I will implement memory-based collaborative filtering by computing cosine similarity, and model-based collaborative filtering by using singular value decomposition.

**The data**: I will use the MovieLens dataset, which contains 100K movie ratings from 944 users and a selection of 1682 movies.

In [2]:
import numpy as np
import pandas as pd

In [3]:
column_names = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('u.data',sep='\t',names=column_names)

In [4]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


get movit names

In [5]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [7]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [8]:
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()
print('number of users: '+str(n_users))
print('number of items: '+str(n_items))

number of users: 944
number of items: 1682


split data into train and test sets

In [17]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,test_size=0.25)

**Memory-based collaborative filtering**: user-item filtering and item-item filtering

user-item filtering: take a particular user, find users that are similar to that user based on similarity of ratings, and recommend those similar users liked. "Users who liked this item also liked..."

item-item filtering: take an item, find users who liked that item, find other item that those users or similar users also liked. It takes items and outputs other items as recommendations. "Users who are similar to you also liked..."

The process is:<br/>
* create a user-item matrix
* train_test_split
* calculate similarities and create similarity matrix. The similarity values for item-item cf are measured by observing all the users who have rated both items, for user-item cf are measured by observing all the items that are rated by both users. Use cosine similarity to calculate distance metric.

create two user-item matrices, one for training and another for testing. It is actually a pivot table with index as user_id and item_id and value as rating.

In [30]:
train_data_matrix = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1] = line[3]

use pairwise distances function from sklearn to calculate the cosine similarity

In [31]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix,metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T,metric='cosine')

Then we are ready to make predictions.<br/>

Applying this formula for user-based cf:
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

sim(uk,ua) is the weight, Xam is the rating of a similar user a for item m. Normalize sim so the ratings stay between 1 and 5. Finally, sum xk that is the average rating for the user that trying to predict.  

Some users may tend to always give high or low ratings for all movies, so the relative difference in the ratings is more important than the absolute values.<br/>
reference: https://medium.com/@tomar.ankur287/user-user-collaborative-filtering-recommender-system-51f568489727

item-based cf:
    <img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [71]:
#np.sum(axis=1), sum of row
def predict(ratings, similarity, type='user'):
    if type =='user':
        mean_user = ratings.mean(axis=1)
        diff = ratings - mean_user[:,np.newaxis]#add one dimension
        pred = mean_user[:,np.newaxis] + similarity.dot(diff)/np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred

* ratings.dimension: (944,1682)
* user_similarity.dim: (944,944)   item_similarity.dim: (1682,1682)
* similarity.dot(diff).dim: (944,1682)  
* np.array([np.abs(similarity).sum(axis=1)]).dim: (1,944)

In [72]:
user_pred = predict(train_data_matrix,user_similarity,'user')

In [73]:
item_pred = predict(train_data_matrix,item_similarity,'item')

Use RMSE to evaluate the model

<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />


In [133]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [134]:
def rmse(pred,test):
    pred = pred[test.nonzero()].flatten()
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred,test))

In [135]:
print('user-based cf rmse: '+str(rmse(user_pred,test_data_matrix)))
print('item-based cf rmse: '+str(rmse(item_pred,test_data_matrix)))

user-based cf rmse: 3.1192716107463996
item-based cf rmse: 3.4463291433047614


Memory-based algorithms are easy to implement and produce reasonable prediction quality. The drawback is that it doesn't address the cold-start problem, which is when new user or new item enters the system.<br/>
reference: https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/

Model-based CF are scalable and can deal with high sparsity level than memory-based models, but also suffer when new users or items don't have any ratings enter into the system.

**Model-based Collaborative Filtering**

Model-based CF is based on matrix factorization(MF). 

First, calculate the sparsity level of MovieLens dataset:

In [142]:
sparsity = round(1-len(df)/float(n_users*n_items),3)
print('sparsity level is '+str(sparsity*100)+'%')

sparsity level is 93.7%


**SVD(singular value decomposition)**

Collaborative filtering can be formulated by apporximating a matrix X by using SVD. The general equation is:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


* u.dim:(m,r) orthogonal matrix
* s.dim: (r,r) diagonal matrix with non-negative real numbers on the diagonal
* v.t: (r,n) orthogonal matrix
* x.dim: (m,n)

Elements on the diagonal of S are singular values of X. Matrix X can be factorized to U,S and V.<br/> 
U matrix: feature vectors corresponding to the users in the hidden feature space.<br/>
V matrix: items in the hidden feature space.

Thus, we can make predictions by taking dot product of U, S, V.T.

Reference: https://stats.stackexchange.com/questions/31096/how-do-i-use-the-svd-in-collaborative-filtering<br/>
https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75

In [144]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

Get SVD components from train matrix, choose k as the number of factors

In [145]:
u,s,vt = svds(train_data_matrix,k=20)
s_diag_matrix = np.diag(s)
x_pred = np.dot(np.dot(u,s_diag_matrix),vt)
print('user-based cf rmse: '+str(rmse(x_pred,test_data_matrix)))

user-based cf rmse: 2.714358065152433
